In [1]:
import pyspark

In [2]:
from pyspark.sql import *

spark = SparkSession.builder\
    .appName("StructuredNetworkWordCount")\
    .getOrCreate()

22/06/25 23:40:40 WARN Utils: Your hostname, Rakibs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.9 instead (on interface en0)
22/06/25 23:40:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/25 23:40:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.format("csv").load("../docker/airflow/data/processed/gtfs/transilien-gtfs-2022-06-24.csv", header=True)

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
relation_schema = StructType()\
    .add("Gare", StringType(), True)\
    .add("Relation", IntegerType(), True)\
    .add("Gare Api", IntegerType(), True)

df_relation = spark.read.format("csv")\
    .options(delimiter=';')\
    .schema(relation_schema)\
    .load("../docker/airflow/data/reference/relation_ordre_RER_B.csv", header=False)

In [5]:
df_relation.show()

+--------------------+--------+--------+
|                Gare|Relation|Gare Api|
+--------------------+--------+--------+
|Aéroport Charles ...|       1|87001479|
|Aéroport CDG 1 (T...|       2|87271460|
|Parc des Expositions|       3|87271486|
|          Villepinte|       4|87271452|
|   Sevran Beaudottes|       5|87271445|
|       Mitry - Claye|       6|87271528|
|Villeparisis - Mi...|       7|87271510|
|         Vert Galant|       8|87271437|
|      Sevran - Livry|       9|87271429|
|    Aulnay-sous-Bois|      10|87271411|
|     Le Blanc-Mesnil|      11|87271478|
|              Drancy|      12|87271403|
|          Le Bourget|      13|87271395|
|La Courneuve - Au...|      14|87271304|
|La Plaine Stade d...|      15|87164798|
|        Gare du Nord|      16|87271007|
| Châtelet les Halles|      17|    null|
|Saint-Michel Notr...|      18|    null|
|          Luxembourg|      19|    null|
|          Port Royal|      20|    null|
+--------------------+--------+--------+
only showing top

In [6]:
df = df.drop("_c0")

In [7]:
df\
    .filter(col("direction")==0)\
    .show(500)

+--------------------+-----------+-------------+---------------+--------------------+--------------+--------------------+-----------+--------------+--------------------+-----------+------------+--------------+---------+
|             trip_id|  trip_line|trip_headsign|trip_short_name|         destination|destination_id|              origin|  origin_id|time_departure|             arrival| arrival_id|time_arrival|time_travelled|direction|
+--------------------+-----------+-------------+---------------+--------------------+--------------+--------------------+-----------+--------------+--------------------+-----------+------------+--------------+---------+
|IDFM:TN:SNCF:31a5...|IDFM:C01743|         JEKI|         JEPI18|   Denfert-Rochereau|    IDFM:71056|Aéroport Charles ...| IDFM:73699|       82610.0|Aéroport CDG 1 (T...| IDFM:73596|     82720.0|         110.0|      0.0|
|IDFM:TN:SNCF:31a5...|IDFM:C01743|         JEKI|         JEPI18|   Denfert-Rochereau|    IDFM:71056|Aéroport CDG 1 (T...

In [8]:
from pyspark.sql.functions import *

# Direction au nord

val = ['IDFM:73596', 'IDFM:73699', 'IDFM:73568', 'IDFM:73547', 'IDFM:73491', 'IDFM:72646', 'IDFM:72648', 'IDFM:72652', 'IDFM:72641', 'IDFM:72598', 'IDFM:72211', 'IDFM:71410', 'IDFM:68916', 'IDFM:69065', 'IDFM:73482', 'IDFM:73439']
df.groupBy("origin", "origin_id", "arrival", "direction")\
    .agg({'time_travelled': 'avg'})\
    .filter(df.direction == 1)\
    .filter(df.origin_id.isin(val))\
    .show(100)

+--------------------+----------+--------------------+---------+-------------------+
|              origin| origin_id|             arrival|direction|avg(time_travelled)|
+--------------------+----------+--------------------+---------+-------------------+
|        Gare du Nord|IDFM:71410|La Plaine Stade d...|      1.0| 262.22222222222223|
|        Gare du Nord|IDFM:71410|Aéroport CDG 1 (T...|      1.0| 1587.7551020408164|
|Villeparisis - Mi...|IDFM:68916|       Mitry - Claye|      1.0|              240.0|
|          Villepinte|IDFM:73547|Parc des Expositions|      1.0|  90.33898305084746|
|     Le Blanc-Mesnil|IDFM:72648|    Aulnay-sous-Bois|      1.0| 105.04273504273505|
|Aéroport CDG 1 (T...|IDFM:73596|Aéroport Charles ...|      1.0| 145.27777777777777|
|              Drancy|IDFM:72652|     Le Blanc-Mesnil|      1.0|              100.0|
|Parc des Expositions|IDFM:73568|Aéroport CDG 1 (T...|      1.0|  291.0169491525424|
|         Vert Galant|IDFM:73482|Villeparisis - Mi...|      1.0| 

In [9]:
# Direction du Sud


df.groupBy("origin", "origin_id", "arrival", "direction") \
    .agg({'time_travelled': 'avg'}) \
    .filter(df.direction == 0) \
    .filter(df.origin_id.isin(val)) \
    .join(df_relation, df.arrival == df_relation.Gare) \
    .join(df_relation.select([col(k).alias(k+'_origin') for k in df_relation.columns]).alias('df2_origin'), df.origin == col('df2_origin.gare_origin')) \


DataFrame[origin: string, origin_id: string, arrival: string, direction: string, avg(time_travelled): double, Gare: string, Relation: int, Gare Api: int, Gare_origin: string, Relation_origin: int, Gare Api_origin: int]

In [188]:
df_sud.join(df_relation, df.arrival == df_relation.Gare, "full").show()

AnalysisException: Resolved attribute(s) arrival#2521 missing from Gare#2830,Relation#2831,Gare Api#2832 in operator !Join FullOuter, (arrival#2521 = Gare#2830).;
!Join FullOuter, (arrival#2521 = Gare#2830)
:- Project
:  +- Sort [Relation#2542 ASC NULLS FIRST], true
:     +- Project [Relation#2542]
:        +- Join Inner, (origin#2518 = Gare#2541)
:           :- Filter (cast(direction#2525 as int) = 0)
:           :  +- Aggregate [origin#2518, origin_id#2519, arrival#2521, direction#2525], [origin#2518, origin_id#2519, arrival#2521, direction#2525, avg(cast(time_travelled#2524 as double)) AS avg(time_travelled)#2807]
:           :     +- Project [trip_id#2512, trip_line#2513, trip_headsign#2514, trip_short_name#2515, destination#2516, destination_id#2517, origin#2518, origin_id#2519, time_departure#2520, arrival#2521, arrival_id#2522, time_arrival#2523, time_travelled#2524, direction#2525]
:           :        +- Relation [_c0#2511,trip_id#2512,trip_line#2513,trip_headsign#2514,trip_sh

In [68]:
df_sud.filter("Relation >= 1 and Relation <= 10").show()

+--------------------+----------+--------------------+---------+-------------------+--------------------+--------+
|              origin| origin_id|             arrival|direction|avg(time_travelled)|                Gare|Relation|
+--------------------+----------+--------------------+---------+-------------------+--------------------+--------+
|Aéroport Charles ...|IDFM:73699|Aéroport CDG 1 (T...|      0.0| 110.09345794392523|Aéroport Charles ...|       1|
|Aéroport CDG 1 (T...|IDFM:73596|Parc des Expositions|      0.0| 280.17543859649123|Aéroport CDG 1 (T...|       2|
|Aéroport CDG 1 (T...|IDFM:73596|        Gare du Nord|      0.0|             1707.4|Aéroport CDG 1 (T...|       2|
|Parc des Expositions|IDFM:73568|          Villepinte|      0.0|  90.17543859649123|Parc des Expositions|       3|
|          Villepinte|IDFM:73547|   Sevran Beaudottes|      0.0| 112.45614035087719|          Villepinte|       4|
|   Sevran Beaudottes|IDFM:73491|    Aulnay-sous-Bois|      0.0| 168.24561403508

In [63]:
result_sud = df_sud.filter("Relation >= 10 and Relation <= 16").agg({"avg(time_travelled)": "sum"}).collect()[0][0]

In [65]:
dt = pendulum.duration(seconds=result_sud)

In [66]:
print(str(dt.in_minutes()) + " minutes")

18 minutes


In [394]:
# b1 = ["87001479", "87271460", "87271486", "87271452", "87271445"] # Aeroport
# b2 = ["87271528", "87271510", "87271437", "87271429"] # Mitry
# central = ["87271411", "87271478", "87271403", "87271395", "87271304", "87164798", "87271007"] # Central

In [31]:
import pendulum
def get_travel_time(gare_depart: str, gare_arrive: str, direction: int) -> str:
    gares: list[str] = compute_inter_station(gare_depart, gare_arrive, direction)
    time = df.groupBy("origin", "origin_id", "arrival", "direction") \
        .agg({'time_travelled': 'avg'}) \
        .filter(df.direction == direction) \
        .join(df_relation, df.arrival == df_relation.Gare) \
        .join(df_relation.select([col(k).alias(k+'_origin') for k in df_relation.columns]).alias('df2_origin'), df.origin == col('df2_origin.gare_origin')) \
        .orderBy("Relation_origin") \
        .filter(col("Gare Api_origin").isin(gares)) \
        .filter(col("Gare Api").isin(gares)) \
        .select(col("origin"), col("arrival"), col("direction"), col("avg(time_travelled)")) \
        .agg({'avg(time_travelled)': 'sum'}).collect()[0][0]
    dt = pendulum.duration(seconds=time)

    # return str(dt.in_minutes()) + " minutes"
    return str(dt.in_minutes()) + " minutes"

In [32]:
def compute_inter_station(gare_depart: str, gare_arrive: str, direction: int) -> list[str]:
    """
    Génère une liste des gare parcourus a partir d'une gare de départ vers une gare d'arrivé
    """

    b1 = ["87001479", "87271460", "87271486", "87271452", "87271445"]  # Aeroport
    b2 = ["87271528", "87271510", "87271437", "87271429"]  # Mitry
    central = ["87271411", "87271478", "87271403", "87271395", "87271304", "87164798", "87271007"]  # Central

    # Detecter l'appartenance de la gare de départ
    res = []
    # Construction des listes
    if direction == 0: # Direction Sud
        if gare_depart in b1:
            res = b1 + central
        elif gare_depart in b2:
            res = b2 + central
        elif gare_depart in central:
            res = central
    elif direction == 1: # Direction nord
        if gare_depart in central:
            central.reverse()
            if gare_arrive in central:
                res = central
            elif gare_arrive in b1:
                b1.reverse()
                res = central + b1
            elif gare_arrive in b2:
                b2.reverse()
                res = central + b2
        elif gare_depart in b1:
            b1.reverse()
            res = b1
        elif gare_depart in b2:
            b2.reverse()
            res = b2

    return res[res.index(gare_depart):res.index(gare_arrive)+1]


In [38]:
print(get_travel_time("87271437","87271007", 0))

21 minutes


In [25]:
direction = 0
# gares: list = compute_inter_station("87271486", "87164798", direction)
gares: list = compute_inter_station("87271411","87271007", 0)
print(gares)
if direction == 1:
    df.groupBy("origin", "origin_id", "arrival", "direction") \
        .agg({'time_travelled': 'avg'}) \
        .filter(df.direction == direction) \
        .join(df_relation, df.arrival == df_relation.Gare) \
        .join(df_relation.select([col(k).alias(k+'_origin') for k in df_relation.columns]).alias('df2_origin'), df.origin == col('df2_origin.gare_origin')) \
        .orderBy("Relation_origin") \
        .filter(col("Gare Api_origin").isin(gares)) \
        .filter(col("Gare Api").isin(gares)) \
        .select(col("origin"), col("arrival"), col("direction"), col("avg(time_travelled)")) \
        .show()
else:
    df.groupBy("origin", "origin_id", "arrival", "direction") \
        .agg({'time_travelled': 'avg'}) \
        .filter(df.direction == direction) \
        .join(df_relation, df.arrival == df_relation.Gare) \
        .join(df_relation.select([col(k).alias(k+'_origin') for k in df_relation.columns]).alias('df2_origin'), df.origin == col('df2_origin.gare_origin')) \
        .orderBy("Relation_origin") \
        .filter(col("Gare Api_origin").isin(gares)) \
        .filter(col("Gare Api").isin(gares)) \
        .select(col("origin"), col("arrival"), col("direction"), col("avg(time_travelled)")) \
        .show()

['87271411', '87271478', '87271403', '87271395', '87271304', '87164798', '87271007']
+--------------------+--------------------+---------+-------------------+
|              origin|             arrival|direction|avg(time_travelled)|
+--------------------+--------------------+---------+-------------------+
|    Aulnay-sous-Bois|     Le Blanc-Mesnil|      0.0| 100.08695652173913|
|     Le Blanc-Mesnil|              Drancy|      0.0| 100.08695652173913|
|              Drancy|          Le Bourget|      0.0|  135.1304347826087|
|          Le Bourget|La Courneuve - Au...|      0.0| 175.04347826086956|
|La Courneuve - Au...|La Plaine Stade d...|      0.0| 120.17391304347827|
|La Plaine Stade d...|        Gare du Nord|      0.0|  269.8333333333333|
+--------------------+--------------------+---------+-------------------+

